In [1]:
import pandas as pd

In [2]:
# Función para muestrear hasta 'n' elementos de cada grupo
def sample_n(group, n=7):
    return group.sample(n=min(len(group), n))

In [3]:
df = pd.read_pickle('../data/api_request_results/connections_catalogue.zip')
df.drop_duplicates(subset=['paperId'], inplace=True)

In [4]:
df_conns = df[~df.openAccessPdf.isna()]

df_conns['pdf_url'] = [i.get('url') for i in df_conns.openAccessPdf]

df_conns.sort_values('db_id', inplace= True)

df_conns = df_conns[['paperId', 'db_id', 'pdf_url', 'con_type']]

df_conns = df_conns.groupby('db_id').apply(sample_n).reset_index(drop=True)

/tmp/ipykernel_29331/913813786.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conns['pdf_url'] = [i.get('url') for i in df_conns.openAccessPdf]
/tmp/ipykernel_29331/913813786.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conns.sort_values('db_id', inplace= True)
/tmp/ipykernel_29331/913813786.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the grou

In [5]:
df_retrieved_full = pd.read_pickle('../data/api_request_results/retrieved_data.zip')

In [6]:
df_retrieved = df_retrieved_full[df_retrieved_full.db_id.isin(df.db_id.unique())]
df_retrieved = df_retrieved[~df_retrieved.openAccessPdf.isna()]

df_retrieved['pdf_url'] = [i.get('url') for i in df_retrieved.openAccessPdf]
df_retrieved['con_type'] = 'original_source'

df_retrieved = df_retrieved[['paperId', 'db_id', 'pdf_url', 'con_type']].reset_index(drop=True)

In [7]:
pdf_catalogue = pd.concat([df_conns, df_retrieved], ignore_index=True)

In [8]:
len(pdf_catalogue)

11897

In [9]:
pdf_catalogue.to_parquet('../data/api_request_results/pdf_catalogue.parquet', index=False)